In [ ]:
#| default_exp utils

In [ ]:
#| export
import string
import uuid

In [ ]:
#| export
def create_nano_id(size=12):
    # Define characters to use (alphanumeric)
    alphabet = string.ascii_letters + string.digits
    
    # Generate UUID and convert to int
    uuid_int = uuid.uuid4().int
    
    # Convert to base62
    result = ""
    while uuid_int:
        uuid_int, remainder = divmod(uuid_int, len(alphabet))
        result = alphabet[remainder] + result
    
    # Pad if necessary and return desired length
    return result[:size]